In [7]:
# Import libraries 
import nltk.data
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
import pandas as pd
import os 
import sys 
#For images analysis
import PIL
from PIL import Image 
import pytesseract 
import pdf2image
from pdf2image import convert_from_path 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
def scanextractor(path,file): 
    """
    Extract text from a scanned PDF and returns a dataframe where each sentences is a new row 
    of the returned dataframe.
    INPUT: -path (string): path that contain the wanted document.
           -name (string): name of the csv document to be used for predictions.
    OUTPUT: - df(dataframe): dataframe containing all the sentences of the text
    """
    PDF_file = (file)
    ''' 
    Part #1 : Converting PDF to images 
    '''
    # Store all the pages of the PDF in a variable 
    pages = convert_from_path(os.path.join(path,file), 500) 
    # Counter to store images of each page of PDF to image 
    image_counter = 1
    # Iterate through all the pages stored above 
    for page in pages: 
    # Declaring filename for each page of PDF as JPG 
    # For each page, filename will be: 
    # PDF page 1 -> page_1.jpg 
    # .... 
    # PDF page n -> page_n.jpg 
        filename = "page_"+str(image_counter)+".jpg"
        # Save the image of the page in system 
        page.save(filename, 'JPEG') 
        # Increment the counter to update filename 
        image_counter = image_counter + 1
    ''' 
    Part #2 - Recognizing text from the images using OCR 
    '''
    # Variable to get count of total number of pages 
    filelimit = image_counter-1
    # Creating a text file to write the output 
    outfile = "{}.txt".format(PDF_file)
    # Open the file in append mode so that 
    # All contents of all images are added to the same file 
    f = open(outfile, "a") 
    # Iterate from 1 to total number of pages 
    for i in range(1, filelimit + 1): 
    # Set filename to recognize text from 
    # Again, these files will be: 
    # page_1.jpg 
    # .... 
    # page_n.jpg 
        filename = "page_"+str(i)+".jpg"
        # Recognize the text as string in image using pytesserct 
        text = str(((pytesseract.image_to_string(Image.open(filename))))) 
        # The recognized text is stored in variable text 
        # Any string processing may be applied on text 
        text = text.replace('-\n', '')
        # Finally, write the processed text to the file. 
        f.write(text) 
        os.remove("page_"+str(i)+".jpg")
    # Close the file after writing all the text. 
    f.close() 
    with codecs.open("{}.txt".format(PDF_file),encoding="latin-1") as f:
        file_data = (f.read())
        text= file_data.replace('\n', '')
        text = text.splitlines()
        text = [el.replace('\xa0',' ') for el in text]
        text = [el.replace('\x92',"'") for el in text]
        text = [el.replace('\x80'," ") for el in text]            
        filename = os.path.basename(file)
        # Remove .pdf from the filename so we can use it as the name of the plot and PNG
        csvname = filename.strip('.txt')
        if text is None:
            print("Error PDF cannot be read")
            sys.exit()
        N = len(text)
        text = [t for t in text if "....." not in t]
        listToStr = ' '.join(map(str, text))
        listToStr = listToStr.replace("’", "'")
        listToStr = listToStr.replace("-",' ')
        data = (tokenizer.tokenize(listToStr))
        new_data = []
        for i in range(int(len(data)/2)):
            j = 2*i
            new_data.append(data[j] + " " + data[j+1])
        df = pd.DataFrame(new_data)
        df = df.rename(columns={0:"Kept sentences"})
        df.to_csv("Table_{}.csv".format(csvname))
        return df

In [3]:
path = input("Step 1: Insert Path Name :")
file = input("Step 2: Insert File Name :")

Step 1: Insert Path Name C:/Users/marco/Desktop/Desktop/OECD/Codes/2014/Afrique/
Step 2: Insert File Name CA II.3.1.1 BENIN.pdf


In [4]:
scanextractor(path,file)

,Kept sentences
0,Annexe 6 Avis Indépendant Développement Durab...
1,Elle prendra la forme d'une densification du r...
2,Ce projet d'extension et de renforcement des r...
3,32Annexe 6 Avis Indépendant Développement Dur...
4,Elle prendra la forme d'une densification du r...
...,...
162,Le classement risque souverain s'étend de RCI ...
163,30S oxouuy I Q[BIOOS UOTPIPSWII}UI |...
164,9p mod Jo sINUeNd Sose}solap sap souonboyy s...
165,Elle prendra la forme d'une densification du r...


In [ ]:
#"C:/Users/marco/Desktop/Desktop/OECD/Codes/2014/Afrique/"
#"CA II.3.1.1 BENIN.pdf"